In [30]:
import pandas as pd
import numpy as np
import spotipy
import json
import plotly.express as px
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
with open('Credenciales.json') as file:
    cred = json.load(file)['Credenciales'][0]
client_id = cred['client_id']
client_secret = cred['client_secret']

In [3]:
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

playlist_id='37i9dQZEVXbO3qyFxbkOE1'
results = sp.playlist(playlist_id)

In [4]:
track_ids = [ item['track']['id'] for item in results['tracks']['items'] ]

spotify_50_mx_df = pd.DataFrame({
    'playlist_id' : playlist_id,
    'playlist_description' : [results['description'] for _ in range(len(track_ids))],
    'playlist_followers' : [results['followers']['total'] for _ in range(len(track_ids))],
    'track_id' : track_ids
})

In [42]:
spotify_50_mx_df['track_id'][0]

'4gzsuuZypVbxs0Af1LSZyB'

In [5]:
dic_data_track ={'track_id':[], 'track_name':[], 'track_release_date': [],
                'explicit':[],'track_popularity':[],
                'album_name':[], 'artist_id':[], 'artist_name':[],
                'artist_followers':[], 'artist_popularity':[],  
                'collaboration_artists_names':[] }

for track_id in spotify_50_mx_df['track_id']:
    # Data track
    data_track = sp.track(track_id)
    dic_data_track['track_id'].append(track_id)
    dic_data_track['track_name'].append(data_track['name'])
    dic_data_track['track_release_date'].append(data_track['album']['release_date'])
    dic_data_track['explicit'].append(data_track['explicit'])
    dic_data_track['track_popularity'].append(data_track['popularity'])
    dic_data_track['album_name'].append(data_track['album']['name'])
    # Data Artist
    data_artist = sp.artist(data_track['artists'][0]['id'])
    dic_data_track['artist_id'].append(data_track['artists'][0]['id'])
    dic_data_track['artist_name'].append(data_track['artists'][0]['name'])
    dic_data_track['artist_followers'].append(data_artist['followers']['total'])
    dic_data_track['artist_popularity'].append(data_artist['popularity'])
    dic_data_track['collaboration_artists_names'].append((',').join([singer_name['name'] for singer_name in data_track['artists']]))


data_track_df = pd.DataFrame.from_dict(dic_data_track)

In [6]:
spotify_50_mx_df = pd.merge(spotify_50_mx_df,
                            data_track_df,
                            on = 'track_id',
                            how = 'left')

In [7]:
features = sp.audio_features(spotify_50_mx_df['track_id'])
features_df=pd.DataFrame.from_dict(features).rename(columns={'id': 'track_id'})
features_df['duration_min']=features_df['duration_ms']/60000

In [8]:
spotify_50_mx_df = pd.merge(spotify_50_mx_df,
                            features_df,
                            on = 'track_id',
                            how = 'left')

In [9]:
spotify_50_mx_df['track_release_date'] = pd.to_datetime(spotify_50_mx_df['track_release_date'], format='%Y-%m-%d',)


In [37]:
spotify_50_mx_df.columns

Index(['playlist_id', 'playlist_description', 'playlist_followers', 'track_id',
       'track_name', 'track_release_date', 'explicit', 'track_popularity',
       'album_name', 'artist_id', 'artist_name', 'artist_followers',
       'artist_popularity', 'collaboration_artists_names', 'danceability',
       'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness',
       'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'uri',
       'track_href', 'analysis_url', 'duration_ms', 'time_signature',
       'duration_min'],
      dtype='object')

In [11]:
df_q3 = spotify_50_mx_df.query("track_release_date.dt.month >= 7")

In [41]:
fig = px.scatter(x=df_q3['artist_popularity'],
                y=df_q3['track_popularity'],
                color=df_q3['artist_followers'])
fig.add_vline(df_q3['artist_popularity'].median())
fig.add_hline(df_q3['track_popularity'].median())
fig.show()

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=afd45396-491d-4511-8873-672a103c2ea0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>